# 误差反向传播
求解梯度有两种方法：
- 上一章中的数值微分法
- 本章中的反向传播法

理解反向传播的过程有两种放方法：
- 数学公式
- 计算图

使用计算图的好处：
- 局部计算：无论计算图有多么复杂，都可以只关注局部的计算
- 使用计算图可以更好地理解反向传播的过程
- 使用计算图可以高效的计算梯度

链式法则：
- 定义：如果某个函数可以由复合函数表示，则该复合函数的导数可以由构成该符合函数的各个函数的导数乘积表示
- 使用公式表达为：$$\frac{\partial f}{\partial x} = \prod \limits_{i=0}^n \frac{\partial f_i}{\partial x_i}$$
- 加法节点的反向传播会将上游的值原封不动的传递给下游
- 乘法节点的反向传播会将输入信号翻转后传给下游

### 乘法层的实现
- 将梯度翻转后传递给下游

In [7]:
# 乘法层反向传播的简单实现
class MulLayer:
    def __init__(self):
        # self.x = None
        # self.y = None
        pass

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y

        return out

    def backward(self, dout): #dout是上一层的梯度
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy

In [8]:
# 测试乘法层的正向、反向传播
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(f'apple_price: {apple_price}')
print(f'tax_price: {price}')

# backforward
# 最终价格自己关于自己的梯度是1
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(f'grad_apple_price:{dapple_price}')
print(f'grad_tax:{dtax}')
print(f'grad_apple:{dapple}')
print(f'grad_apple_num:{dapple_num}')

apple_price: 200
tax_price: 220.00000000000003
grad_apple_price:1.1
grad_tax:200
grad_apple:2.2
grad_apple_num:110.00000000000001


### 加法层的实现
- 将梯度原封不动的传递给下游

In [9]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x, y):
        out = x + y
        return out
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

In [12]:
# 加法层和乘法层的混合测试
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
apple_layer = MulLayer()
orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = apple_layer.forward(apple, apple_num)
orange_price = orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
d_add_apple_price, d_add_orange_price = add_apple_orange_layer.backward(dall_price)
dorange_price, dorange_num = orange_layer.backward(d_add_orange_price)
dapple_price, dapple_num = apple_layer.backward(d_add_apple_price)

# 查看正向传播的过程
print('这里是正向传播的过程')
print(f'apple_price:{apple_price}')
print(f'orange_price:{orange_price}')
print(f'all_price:{all_price}')
print(f'tax_price:{price}')

# 查看反向传播的梯度
print('这里是反向传播的梯度')
print(f'dall_price:{dall_price}')
print(f'dtax:{dtax}')
print(f'd_add_apple_price:{d_add_apple_price}')
print(f'd_add_orange_price:{d_add_orange_price}')
print(f'dorange_price:{dorange_price}')
print(f'dorange_num:{dorange_num}')
print(f'dapple_price:{dapple_price}')
print(f'dapple_num:{dapple_num}')

这里是正向传播的过程
apple_price:200
orange_price:450
all_price:650
tax_price:715.0000000000001
这里是反向传播的梯度
dall_price:1.1
dtax:650
d_add_apple_price:1.1
d_add_orange_price:1.1
dorange_price:3.3000000000000003
dorange_num:165.0
dapple_price:2.2
dapple_num:110.00000000000001


### ReLU层的实现
- 当x>0时，将信号原封不动的传递给下游
- 当x<=0时，传递给下游的信号将在这里终止

In [1]:
class ReluLayer():
    def __init__(self):
        pass
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    def backward(self, dout):
        dout[self.mask]=0
        dx = dout
        return dx

# Sigmoid层的实现
- 上面加法和乘法层的实现，用的是具体的数值来向下游反向传递梯度的，这里的梯度就是具体的值
- Sigmoid这里直接用的函数的导数，把导数当成梯度，向下游传递把导数当成梯度反向传递

In [2]:
import numpy as np
class Sigmoid:
    def __init__(self):
        self.out = None
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    # 这里用了比较多的数学推导
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

## Affine/Softmax层的实现
### Affine

In [5]:
# Affine层的正向传播示例
X = np.array([1.0, 2.0])
W = np.array([[1.0, 3.0, 5.0], [2.0, 4.0, 6.0]])
b = np.array([1.0, 2.0, 3.0])
print(X.shape)
print(W.shape)
print(b.shape)

# 仿射变换
Y = np.dot(X, W) + b
print(Y)
print(Y.shape)

(2,)
(2, 3)
(3,)
[ 6. 13. 20.]
(3,)


In [13]:
# Affine层的定义
import numpy as np
class AffineLayer:
    # 初始化权重和偏置
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    # 可以简单的将点积理解为标量乘法，所以在反向传播时，要将信号翻转
    # 正向传播是dout的形状是一定的，反向传播是当前的的梯度要和当前数据形状一致
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### softmax-with-loss层
- softmax层最后的反向传播的梯度是：$y-t$
- 这么漂亮的结果不是偶然得到的，而是交叉熵和softmax结合的结果，设计出来的

In [20]:
%run ./common/functions.py

In [21]:
# softmax层的定义
from common.functions import softmax
class SoftmaxLayer:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

## 误差反向传播的全貌
### 神将网络学习的全貌
- 前提：选择合适的权重、偏置。神将网络学习的整个过程就是在假设空间中寻找最合适的假设，即寻找最合适的权重、偏置
- 学习的过程分为一下三个部分
  - 选择一个mini-batch，然后计算整个batch的loss，最后会用权重和偏置的形式表达出来
  - 计算每一个参数的梯度：给每一参数加上delta，然后f(x+delta) - f(x-delta) / 2delta就可以计算出来每个参数在这个假设空间下的梯度
  - 使用当前的值减去学习率和梯度的乘积，更新参数
  - 重复上面的三个过程，一定会让问题得到收敛，最后的loss在不断的减小
- 上面这个过程虽然简单，但是在计算梯度时，开销太大了，具体表现为每次更新梯度时，有多少个参数，都要计算多少次，所以这一章中使用的是计算图来计算梯度